In [11]:
import pandas as pd
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2")

In [12]:
df = pd.read_csv('prospectuses_data.csv')

# create a df copy with rows that do not hace "failed parsing" in the Section ID column
df = df[df['Section ID'] != "failed parsing"]
df.head(2)

,Prospectus ID,Original Filename,Section ID,Section Title,Subsection ID,Subsection Title,Subsubsection ID,Subsubsection Title,Subsubsection Text,Market Dynamics - a,Market Dynamics - b,Market Dynamics - c,LLM Answer,Evidence Text,Parsing Error
3,4,FR0014001YE4.pdf,1,RISK FACTORS,1.1,1. Risks related to the Issuer,1.1.1,NaN,Risk factors relating to the Issuer and the Gr...,NaN,NaN,NaN,NaN,NaN,NaN
4,4,FR0014001YE4.pdf,1,RISK FACTORS,1.2,2. Risks related to the Bonds 2.1 Risks relati...,1.2.1,2.1.1 The Bonds may be redeemed prior to maturity,The Issuer reserves the right to purchase Bond...,NaN,NaN,NaN,NaN,NaN,NaN


### System Prompt
You are an expert in analyzing bond prospectuses and identifying specific risk factors. Provide clear "Yes" or "No" answers to questions.

### User Prompt
Given the following excerpt from a bond prospectus:

Title: [Subsubsection Title]

Text: [Subsubsection Text]

Please determine whether the text indicates the presence of the following characteristics. For each, answer "Yes" or "No".

a. Exposure to cyclical products

b. Impact of demographic and structural trends

c. Seasonal industry volatility

Please provide your answer in the following JSON format:

{
  "a": "Yes" or "No",
  "b": "Yes" or "No",
  "c": "Yes" or "No"
}

In [20]:
def analyze_prospectus_row(row):
    # System and user prompts
    system_prompt = "You are an expert in analyzing bond prospectuses and identifying specific risk factors. Provide clear 'Yes' or 'No' answers to questions."
    
    # Format the user prompt using the row's data
    prompt = f"""
    Given the following excerpt from a bond prospectus: {row['Subsubsection Title']} \n {row['Subsubsection Text']}

    Please determine whether the text indicates a risk of exposure to cyclical products. For each, answer "Yes" or "No".

    a. Exposure to cyclical products

    b. Impact of demographic and structural trends

    c. Seasonal industry volatility

    Please provide your answer in the following JSON format:

    {{
      "a": "Yes" or "No",
      "b": "Yes" or "No",
      "c": "Yes" or "No"
    }}
    """
    # Run the prompt through the model
    response = llm(prompt=prompt)           
    
    # Return response as a dictionary for easy insertion into DataFrame
    return response

In [21]:
# create a copy of df with only the first 10 rows
df_subset = df.head(10)

In [26]:
# Apply the function row by row and store results in a new DataFrame column
df['analysis'] = df.apply(analyze_prospectus_row, axis=1)
df[['Subsubsection Title', 'Subsubsection Text', 'analysis']].head(2)

,Subsubsection Title,Subsubsection Text,analysis
3,NaN,Risk factors relating to the Issuer and the Gr...,"Based on the provided excerpt, here is the det..."
4,2.1.1 The Bonds may be redeemed prior to maturity,The Issuer reserves the right to purchase Bond...,"Here is the response in JSON format:\n\n{\n ""..."


In [30]:
df['analysis'].values

array(['Based on the provided excerpt, here is the determination of risk factors related to cyclical products and other industry characteristics:\n\n\n{\n  "a": "No",\n  "b": "Yes",\n  "c": "No"\n}',
       'Here is the response in JSON format:\n\n{\n  "a": "No",\n  "b": "No",\n  "c": "No"\n}',
       '{\n  "a": "No",\n  "b": "No",\n  "c": "Yes"\n}',
       '{\n  "a": "No",\n  "b": "No",\n  "c": "No"\n}',
       'Here is the response in the requested JSON format:\n\n{\n  "a": "No",\n  "b": "Yes",\n  "c": "No"\n}',
       'Given the excerpt, here is the response in the requested JSON format:\n\n\n{\n  "a": "No",\n  "b": "Yes",\n  "c": "No"\n}',
       '{\n  "a": "No",\n  "b": "No",\n  "c": "No"\n}',
       'Based on the provided excerpt, here is the determination of risks related to cyclical products and other factors:\n\n\n{\n  "a": "Yes",\n  "b": "No",\n  "c": "No"\n}',
       'Based on the provided excerpt, I would answer as follows:\n\n{\n  "a": "No",\n  "b": "No",\n  "c": "No"\n}\n